# Title 
## 4. Modeling

### Import statements and files

In [2]:
import numpy as np
import pandas as pd
import sklearn


In [3]:
hatemaps = pd.read_csv('/Users/gemma/Documents/data science/fc-hatemaps.csv')
crimes_by_state = pd.read_csv('/Users/gemma/Documents/data science/fc-crimes_by_state.csv')
state_totals = pd.read_csv('/Users/gemma/Documents/data science/fc-hate-state_totals.csv')
by_city_only = pd.read_csv('/Users/gemma/Documents/data science/fc-hate-by_city.csv')

### Feature engineering

First, some ideas of features that I had. Later, I'll use some unsupervised methods.

* Number of unique cities per state with at least 1 reported hate crime
* Max count of hate groups in any 1 city in each state
* Number of unique cities per state with at least 1 hate group
* Binary indicators of 5 largest hate groups presence in each city / state

In [4]:
# number of unique cities per state with at least 1 reported hate crime

state_totals = pd.merge(state_totals, 
                        by_city_only.groupby(['Year','State'])['Agency name'].nunique().reset_index().rename(index=str,
                            columns={'Agency name': 'Number of cities with 1+ hate crime'}), 
                        on=['Year','State'])

In [5]:
# max count of hate groups in 1 city in each state

state_totals = pd.merge(state_totals, hatemaps.groupby(['Year','State',
                  'City'])['Group Name'].count().groupby(['Year',
                 'State']).max().reset_index().rename(index=str,columns={'Group Name': 
                    'Largest num of hate groups in any 1 city'}),
                        on=['Year','State'])


In [6]:
# count of unique cities with 1 or more hate groups in each state per year

state_totals = pd.merge(state_totals, 
                        hatemaps.groupby(['Year',
                            'State'])['City'].nunique().reset_index().rename(index=str,
                            columns={'City': 'Number of cities with 1+ hate groups'}), 
                        on=['Year','State'])

In [8]:
# for the cumulatively largest 5 hate groups (by count) in each category, 
# binary indicators of whether or not they exist in each state

# first get the names of the hate groups
largest_five = []

for htype in hatemaps['Hate Type'].unique():

    templist = hatemaps[hatemaps['Hate Type'] == htype].groupby(['Group Name'])['Group Name'].count().sort_values(ascending=False).head(5).index.tolist()

    for gr_name in templist:
        # add it to the list of largest five
        largest_five.append(gr_name)
        # also add a column to the hatemaps
        hatemaps[gr_name] = 0


In [9]:
# then add the names to the dataframe

for i, name in enumerate(hatemaps['Group Name'].tolist()):
    if name in largest_five:
        hatemaps.loc[i, name] =1

In [10]:
# add them as features to the states table

state_totals = pd.merge(state_totals, 
                        hatemaps.groupby(['Year','State'])[largest_five].sum().reset_index(), 
                        on=['Year','State'])

In [12]:
# add them as features to the city table as well

by_city_only = pd.merge(by_city_only, hatemaps, how="outer",  
                        right_on=['Year','State','City'], 
         left_on=['Year','State', 'Agency name']).fillna(0)

## feature engineering with unsupervised learning

* NLP - topic extraction
* Autoencoders?? (neural networks)
* clusters with mean shift

Using NLP I'll analyze the names of the hategroups and design features that will then be added to the tables above.

* keyword extraction
* topics

In [31]:
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer

import nltk
from nltk.corpus import stopwords
import string

import networkx as nx

from gensim.test.utils import datapath
from gensim.models.word2vec import Text8Corpus
from gensim.models.phrases import Phrases, Phraser

In [20]:
# get all of the hate group names

hgnames = list(set(hatemaps['Group Name'].tolist()))

hgnames_string = ".".join(hgnames)

In [25]:
nlp = spacy.load('en')

# parse the names
hgp = nlp(hgnames_string)

In [51]:
# Dividing the text into sentences and storing them as a list of strings.
sentences=[]
for span in hgp.sents:
    # go from the start to the end of each span, returning each token in the sentence
    # combine each token using join()
    sent = ''.join(hgp[i].string for i in range(span.start, span.end)).strip()
    sentences.append(sent)

# Creating the tf-idf matrix.
counter = TfidfVectorizer(lowercase=False, 
                          stop_words='english',
                          ngram_range=(1, 1), 
                          analyzer=u'word', 
                          max_df=.5, 
                          min_df=1,
                          max_features=None, 
                          vocabulary=None, 
                          binary=False)

#Applying the vectorizer
data_counts=counter.fit_transform(sentences)

In [52]:
stop = stopwords.words('english') + list(string.punctuation) + ["''", "``" , "--", "n't", "'ve", "'s"]


sentence_stream = [[i.lower() for i in nltk.word_tokenize(sent) if i.lower() not in stop]  for sent in sentences]
bigram = Phrases(sentence_stream, min_count=1, threshold=3, delimiter=b' ')
bigram_phraser = Phraser(bigram)
tokens_ = bigram_phraser[sentence_stream]

bigrams_t = tokens_

bigrams_o = [i for j in bigrams_t for i in j]
print(bigrams_o[:25])

['white', 'revolution.appalachian', 'knights', 'ku klu', 'klan.northern', 'california', 'aryan', 'volk.south', 'africa', 'project.stone', 'kingdom', 'ministries.h.o.m.e', 'heteroseuals', 'organized moral', 'environment.united', 'patriots', 'associates.american', 'constitution', 'center.the', 'fitzgeral', 'griffin foundation.the', 'political', 'cesspool.freedom', '14.church', 'sons']


In [53]:
# Removing stop words and punctuation, then getting a list of all unique words in the text
hgp_filt_2 = [word for word in bigrams_o if word not in stop]
words2=set(hgp_filt_2)

#Creating a grid indicating whether words are within 4 places of the target word
adjacency2=pd.DataFrame(columns=words2,index=words2,data=0)

#Iterating through each word in the text and indicating which of the unique words are its neighbors
for i,word in enumerate(bigrams_o):
    # Checking if any of the word's next four neighbors are in the word list 
    if any([word == item for item in hgp_filt_2]):
        # Making sure to stop at the end of the string, even if there are less than 
        # four words left after the target.
        end=max(0,len(bigrams_o)-(len(bigrams_o)-(i+6)))
        # The potential neighbors.
        nextwords=bigrams_o[i+1:end]
        # Filtering the neighbors to select only those in the word list
        inset=[x in hgp_filt_2 for x in nextwords]
        neighbors=[nextwords[i] for i in range(len(nextwords)) if inset[i]]
        # Adding 1 to the adjacency matrix for neighbors of the target word
        if neighbors:
            adjacency2.loc[word,neighbors]=adjacency2.loc[word,neighbors]+1

print('done!')


done!


In [54]:

# Running TextRank
nx_words2 = nx.from_numpy_matrix(adjacency2.as_matrix())
ranks2=nx.pagerank(nx_words2, alpha=.85, tol=.00000001)

# Identifying the five most highly ranked keywords
ranked2 = sorted(((ranks2[i],s) for i,s in enumerate(words2)),
                reverse=True)
print(ranked2[:5])


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


[(0.021299311111950592, 'knights ku'), (0.020347281589460944, 'ku klu'), (0.011694463153713325, 'klu'), (0.00870997645071624, 'white knights'), (0.007970037497157042, 'american')]


In [38]:
trigram = Phrases(tokens_, min_count=1, threshold=2, delimiter=b' ')
trigram_phraser = Phraser(trigram)
tokens__ = trigram_phraser[tokens_]

all_words = [i for j in tokens__ for i in j]
print(all_words[:50])

['white', 'revolution.appalachian', 'knights', 'ku klu', 'klan.northern', 'california', 'aryan', 'volk.south', 'africa', 'project.stone', 'kingdom', 'ministries.h.o.m.e', 'heteroseuals', 'organized moral', 'environment.united', 'patriots', 'associates.american', 'constitution', 'center.the', 'fitzgeral', 'griffin foundation.the', 'political', 'cesspool.freedom', '14.church', 'sons', 'yhvh', 'reformed.bob', 'underground', 'graduate', 'seminar/bugs.traditional', 'values', 'coalition.catholic', 'action', 'resource', 'center.hated', 'the.new', 'beginnings.aryan', 'nations', 'offshoot', '.council', 'social', 'economic', 'studies.creativity', 'alliance', 'the.colorado', 'alliance', 'immigration', 'reform.desastrious', 'records.original', 'knight riders knights ku']


In [39]:
# Removing stop words and punctuation, then getting a list of all unique words in the text
gatsby_filt_2 = [word for word in all_words if word not in stop]
words2=set(gatsby_filt_2)

#Creating a grid indicating whether words are within 4 places of the target word
adjacency2=pd.DataFrame(columns=words2,index=words2,data=0)

#Iterating through each word in the text and indicating which of the unique words are its neighbors
for i,word in enumerate(all_words):
    # Checking if any of the word's next four neighbors are in the word list 
    if any([word == item for item in gatsby_filt_2]):
        # Making sure to stop at the end of the string, even if there are less than 
        # four words left after the target.
        end=max(0,len(all_words)-(len(all_words)-(i+6)))
        # The potential neighbors.
        nextwords=all_words[i+1:end]
        # Filtering the neighbors to select only those in the word list
        inset=[x in gatsby_filt_2 for x in nextwords]
        neighbors=[nextwords[i] for i in range(len(nextwords)) if inset[i]]
        # Adding 1 to the adjacency matrix for neighbors of the target word
        if neighbors:
            adjacency2.loc[word,neighbors]=adjacency2.loc[word,neighbors]+1

print('done!')


done!


In [42]:

# Running TextRank
nx_words2 = nx.from_numpy_matrix(adjacency2.as_matrix())
ranks2=nx.pagerank(nx_words2, alpha=.85, tol=.00000001)

# Identifying the five most highly ranked keywords
ranked2 = sorted(((ranks2[i],s) for i,s in enumerate(words2)),
                reverse=True)
print(ranked2[:5])


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


[(0.009917534666537255, 'knights ku klu'), (0.007023840180168423, 'american'), (0.007002710179014081, 'white knights ku klu'), (0.0066533633994273434, 'state'), (0.006337600714110033, 'ku klu')]


Extracting keywords with both bigram and trigram resulted in very similar results, perhaps unsurprisingly, considering the sizes of the hate groups from the data analysis section. They show that the most common key words are variations of "ku klux klan" for the most part, and also "white", "american", and "state".

#### Clustering 

I am going to use mean shift because it won't assign all points to a cluster unless it is close enough, and because the algorithm doesn't assume that each cluster has to be the same size.

In [59]:
state_totals.rename(index=str,columns={'Race / Ethnicity / Ancestry': 'REA'}, inplace=True)

In [61]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

In [62]:
# first, splitting the data to x and y, then normalizing it

outcome_vs = ['REA','Religion', 'Disability','Gender','Gender Identity', 'Sexual orientation']
   
X = state_totals.drop(['REA','Religion',
                       'Disability','Gender','Gender Identity', 
                       'Sexual\norientation', 'State', 'State_name'], 1).fillna(0)
Y = state_totals[outcome_vs].fillna(0)

X_train_mo, X_test_mo, y_train_mo, y_test_mo = train_test_split(
    X, Y, test_size=0.6, random_state=0)

In [85]:
# then scaling the data 

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0, 1))

x_train_scaled = scaler.fit_transform(X_train_mo)


In [86]:
from sklearn.cluster import MeanShift, estimate_bandwidth

# Here we set the bandwidth. This function automatically derives a bandwidth
# number based on an inspection of the distances among points in the data.
bandwidth = estimate_bandwidth(x_train_scaled, quantile=0.2, n_samples=500)

# Declare and fit the model.
ms = MeanShift(bandwidth=bandwidth, bin_seeding=True)
ms.fit(x_train_scaled)

# Extract cluster assignments for each data point.
labels = ms.labels_

# Coordinates of the cluster centers.
cluster_centers = ms.cluster_centers_

# Count our clusters.
n_clusters_ = len(np.unique(labels))

print("Number of estimated clusters: {}".format(n_clusters_))

Number of estimated clusters: 7


In [87]:
clusters_s = pd.Series(ms.labels_.tolist())

In [88]:
X_train_mo.insert(loc=0, column='meanshift', value=clusters_s.values)

ValueError: cannot insert meanshift, already exists

In [ ]:

#x_test_scaled = scaler.fit_transform(X_test_mo)
X_scaled = scaler.fit_transform(X)

In [89]:
bandwidth = estimate_bandwidth(x_train_scaled, quantile=0.2, n_samples=500)

# Declare and fit the model.
ms = MeanShift(bandwidth=bandwidth, bin_seeding=True)
ms.fit(X_scaled)

# Extract cluster assignments for each data point.
labels = ms.labels_

clusters_X = pd.Series(ms.labels_.tolist())

X.insert(loc=0, column='meanshift', value=clusters_X.values)

In [90]:
X_train_mo, X_test_mo, y_train_mo, y_test_mo = train_test_split(
    X, Y, test_size=0.6, random_state=0)

Next, some feature selection with KBest and feature importance...

### Feature selection

### Supervised learning with state data

## Random forest

with multi output

In [80]:
# random forest

from sklearn import ensemble
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split


In [81]:
rfr = ensemble.RandomForestRegressor(warm_start=True)

In [91]:
rfr = ensemble.RandomForestRegressor(n_estimators=100, warm_start=True)

rfr.fit(X_train_mo, y_train_mo)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=True)

In [92]:
rfr.score(X_test_mo,y_test_mo)

0.8840495966585692

In [94]:
test_score_rfr_mo = cross_val_score(rfr, X_test_mo, y_test_mo, cv=10)

print('Testing score: {} +/- {}'.format(np.mean(test_score_rfr_mo),np.std(test_score_rfr_mo)))

Testing score: 0.9012715983443552 +/- 0.04591303181433302


### KNN with multi output

Using the already scaled data, above

In [95]:
# getting the best # for n
from sklearn.model_selection import GridSearchCV

from sklearn.neighbors import KNeighborsRegressor

knn = KNeighborsRegressor()

params = {'n_neighbors':[2,3,4,5,6,7,8,9], 'weights': ['uniform', 'distance']}

model = GridSearchCV(knn, params, cv=5)

In [96]:
model.fit(x_train_scaled,y_train_mo)
model.best_params_

{'n_neighbors': 3, 'weights': 'distance'}

In [98]:
knn = KNeighborsRegressor(n_neighbors=3, weights='distance')

knn.fit(x_train_scaled,y_train_mo)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=3, p=2,
          weights='distance')

In [99]:
knn.score(x_test_scaled,y_test_mo)

ValueError: query data dimension must match training data dimension

### SVR with Multi Output

In [101]:
from sklearn.svm import SVR

from sklearn.multioutput import MultiOutputRegressor

svr_mor = MultiOutputRegressor(SVR())

svr_mor.fit(X_train_mo,y_train_mo)

MultiOutputRegressor(estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False),
           n_jobs=1)

In [102]:
svr_mor.score(X_test_mo,y_test_mo)

-0.07999107745592449

In [103]:
test_score_svrm = cross_val_score(svr_mor, X_test_mo, y_test_mo, cv=10)

print('Testing score: {} +/- {}'.format(np.mean(test_score_svrm),np.std(test_score_svrm)))

Testing score: -0.10129809028102743 +/- 0.05819644645096418


### Supervised learning with City data

In [114]:
by_city_only.head()

,Unnamed: 0_x,Year,State_name,Agency name,REA,Religion,Disability,Gender,Gender Identity,Sexual orientation,...,msr productions,isd records,micetrap distribution,desastrious records,tightrope,tradition in action,slaves of the immaculate heart of mary,catholic counterpoint,culture wars/fidelity press,omni christian book club
0,0.0,2006,Alabama,Atmore,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,2006,Alaska,Anchorage,4.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2.0,2006,Arizona,Apache Junction,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3.0,2006,Arizona,Bullhead City,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4.0,2006,Arizona,Chandler,7.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [111]:
# for multi output regression
X_city = by_city_only.drop(['REA','Religion', 'Disability','Gender',
                            'Gender Identity',
                       'Sexual orientation', 'State', 'State_name',
                            'Agency name','Unnamed: 0_x', 
                           'Group Name_x', 'City_x',
       'Hate Type_x', 'Unnamed: 0_y', 'Group Name_y', 'City_y',
       'Hate Type_y',], 1).fillna(0)
Y_city = by_city_only[outcome_vs].fillna(0)


In [112]:

X_train_city_mo, X_test_city_mo, y_train_city_mo, y_test_city_mo = train_test_split(
    X_city, Y_city, test_size=0.6, random_state=0)

In [113]:
# scaled for KNN

scaler = MinMaxScaler(feature_range=(0, 1))

x_train_city_scaled = scaler.fit_transform(X_train_city_mo)

x_test_city_scaled = scaler.fit_transform(X_test_city_mo)



### Random forest

In [115]:
rfr = ensemble.RandomForestRegressor(warm_start=True)

In [116]:
rfr.fit(X_train_city_mo, y_train_city_mo)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=True)

In [117]:
rfr.score(X_test_city_mo, y_test_city_mo)

0.16073519065653813

In [119]:

test_score_city_rfr = cross_val_score(rfr, X_test_city_mo, y_test_city_mo, cv=10)
print('{} +/- {}'.format(np.mean(test_score_city_rfr), np.std(test_score_city_rfr)))



0.1713418025924668 +/- 0.016816419672536793


### KNN Regression

In [120]:
knn = KNeighborsRegressor(n_neighbors=3, weights='distance')

In [123]:
knn.fit(x_train_city_scaled,y_train_city_mo)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=3, p=2,
          weights='distance')

In [125]:
knn.score(x_test_city_scaled,y_test_city_mo)

0.008610934920477905

In [126]:
test_score_city_knn = cross_val_score(knn, x_test_city_scaled, y_test_city_mo, cv=10)

print('{}  +/-  {}'.format(np.mean(test_score_city_knn), np.std(test_score_city_knn)))

-0.04458093968079231  +/-  0.18534272278545533


### SVR

In [ ]:
from sklearn.svm import SVR

svr_mor = MultiOutputRegressor(SVR())

svr_mor.fit(X_train_city_mo,y_train_city_mo)

In [ ]:
svr_mor.score(X_test_city_mo, y_test_city_mo)

In [ ]:
test_score_city_svr = cross_val_score(svr_mor, X_test_city_mo, y_test_city_mo,cv=10)
print('{}  +/-  {}'.format(np.mean(test_score_city_svr), np.std(test_score_city_svr)))